# Description of the algorithm :
This algorithm aims to browse all live swimming competitions on this website:  https://www.liveffn.com/cgi-bin/liste_live.php

For each competition, the script retrieves the data from the page using requests. Then, it parse the data using beautiful soup. For each competition, we check the presence of a particular club: here it is the swimming club named "TOEC".
If a swimmer of "TOEC" is present at this competition, it retrieves the data: The results and the commitments. Then, the script arranges all these data to create a JSON file readable and ordered. At the end of all the competitions, the script exports the Results and the Commitments in Json and Javascript files. 

In [ ]:


from ctypes import pointer


from bs4 import BeautifulSoup as bs
import re
import json
from os import path
import requests



#url='https://www.liveffn.com/cgi-bin/liste_live.php' POUR COMEPE EN COURS
url = "https://www.liveffn.com/cgi-bin/liste_live.php?page=recemment_termine"
page=requests.request("GET",url)
soup = bs(page.text, 'html.parser')
contents = soup.find_all(class_= 'containeur_niveauN')

RegionalLink = []
#print(soup.prettify())
index = 0
for link in soup.find_all(class_= 'containeur_niveauI'):
    child1 = link.findChildren(class_= 'visuel_img')
    for a in child1:
        children = a.findChildren('a')
        RegionalLink.append(children[0].get('href')) 
        index = index+1
        #print(children[0].get('href'))
for link in soup.find_all(class_= 'containeur_niveauN'):
    child1 = link.findChildren(class_= 'visuel_img')
    for a in child1:
        children = a.findChildren('a')
        RegionalLink.append(children[0].get('href')) 
        index = index+1
        #print(children[0].get('href'))
for link in soup.find_all(class_= 'containeur_niveauR'):
    child1 = link.findChildren(class_= 'visuel_img')
    for a in child1:
        children = a.findChildren('a')
        RegionalLink.append(children[0].get('href')) 
        index = index+1
for link in soup.find_all(class_= 'containeur_niveauD'):
    child1 = link.findChildren(class_= 'visuel_img')
    for a in child1:
        children = a.findChildren('a')
        RegionalLink.append(children[0].get('href')) 
        index = index+1




def getname(link):
    page=requests.request("GET",link)
    soup = bs(page.data, 'html.parser')
    contents = soup.find_all(class_= 'titre')
    #print(contents[0].get_text())
    return contents[0].get_text()
 

def getProglink(link):
    page=requests.request("GET",link)
    soup = bs(page.text, 'html.parser')
    contents = soup.find_all(class_= 'sdt_box')
    if len(contents) == 0 :
        print("######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######")
        return None
    else :
        #print(contents[1].findChildren('a')[0].get('href'))
        return (contents[1].findChildren('a')[0].get('href'))
    
    
def getstructlink(link):
    page=requests.request("GET",link)
    soup = bs(page.text, 'html.parser')
    contents = soup.find_all(class_= 'navigation')
    if len(contents) == 0 :
        print("No data yet")
        return None
    else :
        #print(contents[0].findChildren(class_ = 'tooltip')[3].get('href'))
        return(contents[0].findChildren(class_ = 'tooltip')[3].get('href'))
        print("\n")
    
    
def ListParticipants(link):
    Participants = []
    page=requests.request("GET",link)
    soup = bs(page.text, 'html.parser')
    contents = soup.find_all(class_= 'survol')

    if len(contents) == 0 :
        print("###No swimmer###")
    else :
        for item in contents:
            objet = {}
            if item.findChildren(class_ = 'resNumStructure') == []:
                num = '00000000000'
                nom = objet['nom'] = item.findChildren(class_ = 'underline')[0].get_text()
            else : 
            # print(item.findChildren(class_ = 'resNumStructure')[0].get_text())
                num = item.findChildren(class_ = 'resNumStructure')[0].get_text()
                nom = objet['nom'] = item.findChildren(class_ = 'underline')[0].get_text()
            lien = objet['nom'] = item.findChildren(class_ = 'underline')[0].get('href')
            num = re.sub("\t|\r|\n","",num)
            nom = re.sub("\t|\r|\n","",nom)
            objet["num"] = num
            objet["nom"] = nom
            objet["lien"] = lien
            Participants.append(objet)
        #print (Participants)
        return Participants
    

def getsdetailstructlink(numclub,participants):
    if participants == None:
        return 0
    for item in participants:
        numero = item["num"][0:11]
        #print (numero)
        if numero == numclub:
            #print("OOOOOKKKKKKKKKKKKK")
            #print(item["nom"])
            
            #####INFO COMPETE###
            index = -1
            Database = []
            compete = {}
            objet = {}
            nages = []
            nageurs = []
            lien = item["lien"]
            page=requests.request("GET",lien)
            soup = bs(page.text, 'html.parser')
            content = soup.find_all(id= "header")
            infos = {}

            nom =content[0].findChildren(class_ = "titre" )[0].get_text()
            nom = re.sub("\t|\r|\n","",nom)
            infos["nom"] = nom
            compete['infos'] = infos
            
            ##
            
            lien = item["lien"]
            #print(lien)
            page=requests.request("GET",lien)
            soup = bs(page.text, 'html.parser')
            contents = soup.find_all(class_= 'tableau')

            
                                   
                    
            for item in (contents[0].findChildren("tr")) :

                if item.findChildren('td')!=[] :
                    if item.findChildren(class_ = "resStructureIndividu2" ) or item.findChildren(class_ = "resStructureIndividu1" ) :
                        #print(item.findChildren('td')[0].get_text())
                        if index == -1 :
                            None
                        else :
                            objet['nages'] = nages
                            nageurs.append(objet)
                            objet = {}
                            nages = []
                        nom = item.findChildren('td')[0].get_text()
                        nom = re.sub("\t|\r|\n","",nom)
                        objet['nom'] =nom
                        index = index+1 

                        #print (index)
                    elif  item.findChildren(class_ = "resStructureRelayeur" ) == [] :
                        None
                    else :
                        #print (item.findChildren('td')[0].get_text())
                        nages.append(item.findChildren('td')[0].get_text())
            
            objet['nages'] = nages
            nageurs.append(objet)
            compete["nageurs"] = nageurs
            Database.append(compete)
            
            
            file = "data.json"
            entry = {'carl': 33}  
            #jsonFile = open("data.json", "w")
            with open(file,"r+") as file :
                data = json.load(file)
                data.append(Database)
                file.seek(0)
                json.dump(data, file, indent=4,)
           

            #print(Database)    
            #export = json.dumps(Database,indent=4)
            #print (export)
            #export = "data = "+export
            #jsonFile = open("data.json", "w")
           # json.dumps(export, jsonFile,indent=4,)
            #jsonFile.write(export)
            #jsonFile.close()
           # with open('json_data.json', 'w') as jsonfile:
           #      json.dump(export, jsonfile)
                 

            return (Database)   
            





def getresult(numclub,participants,lienacceuil):
    isTOEC = 0
    page=requests.request("GET",lienacceuil)
    soup = bs(page.text, 'html.parser')
    contents = soup.find_all(class_= 'sdt_menu')
    if len(contents) == 0 :
        print("######!!!!!###### Impossible to get data, web page different from standars ########!!!!!!!!!#######")

        return None
    else :
        #print(contents[0].findChildren('li')[2].findChildren('a')[0].get('href'))
        if contents[0].findChildren('li')[2].findChildren(class_ = "sdt_link")[0].get_text() != "Résultats":
            linkresult = (contents[0].findChildren('li')[3].findChildren('a')[0].get('href'))
        else :
            linkresult = (contents[0].findChildren('li')[2].findChildren('a')[0].get('href'))
    
    
    page=requests.request("GET",linkresult)
    soup = bs(page.text, 'html.parser')
    resultPage = soup.find_all(class_= 'survol')
    #print(resultPage)
    if len(resultPage) == 0 :
        print("######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######")

        return None
    else :
      #  print(resultPage[0].findChildren('a')[0].get('href'))
        if resultPage[0].findChildren('a') == []:
            print ("echec d'acces aux resultats pour le lien :"+linkresult )
            return 0
        linkresult =  (resultPage[0].findChildren('a')[0].get('href'))
        page=requests.request("GET",linkresult)
        soup = bs(page.text, 'html.parser')
        contents = soup.find_all(class_= 'navigation')
        if len(contents) == 0 :
            print("Resultat pas encore dispo")
            return None
        else :
        #print(contents[0].findChildren(class_ = 'tooltip')[3].get('href'))
            linkresult = (contents[0].findChildren(class_ = 'tooltip')[2].get('href'))
            print("\n")
            
            ######## GET RESULT ######
            ListeClub = (ListParticipants(linkresult))
            if ListeClub == None :
                print("Liste club est null")
                return None
            else :
                for item in ListeClub:
                    numero = item["num"][0:11]
                #print (numero)
                    if numero == numclub or (item["nom"] == "DAUPHINS TOULOUSE OEC"):
    
                        print("###################TOEC FIND#########################")
                        print("The swimming club TOEC participate at "+item["nom"]+" with the link :"+item["lien"])
                        print("###################TOEC FIND#########################")
                        isTOEC = 1
                        lien = item["lien"]
                        
                                            #info compète
                        compete = {}
                        page=requests.request("GET",lien)
                        soup = bs(page.text, 'html.parser')
                        content = soup.find_all(id= "header")
                        infos = {}

                        nom =content[0].findChildren(class_ = "titre" )[0].get_text()
                        nom = re.sub("\t|\r|\n","",nom)
                        infos["nom"] = nom
                        compete['infos'] = infos
                        
                        ##
                        
                        page=requests.request("GET",lien)
                        soup = bs(page.text, 'html.parser')
                        contents = soup.find_all(class_= 'tableau')
                        index = -1
                        Database = []
                        
                        
                        objet = {}
                        nages = []
                        OneNage = {}
                        nageurs = []
                        print(" ")
                        print("Getting swimmer data ...")
                        for item in (contents[0].findChildren("tr")) :

                            if item.findChildren('td')!=[] :
                                if item.findChildren(class_ = "resStructureIndividu2" ) or item.findChildren(class_ = "resStructureIndividu1" ) :
                                    #print(item.findChildren('td')[0].get_text())
                                    if index == -1 :
                                        None
                                    else :
                                        nageurs.append(objet)
                                        objet['nages'] = nages
                                        objet = {}
                                        nages = []
                                        Onenage = {}
                                    nom = item.findChildren('td')[0].get_text()
                                    nom = re.sub("\t|\r|\n","",nom)
                                    objet['nom'] =nom
                                    index = index+1 
                                    print(nom+" ", end='')

                                    #print (index)
                                elif  item.findChildren(class_ = "resStructureRelayeur" ) == [] :
                                    None
                                else :
                                    #print (item.findChildren(class_ = 'underline')[0].get_text())
                                    OneNage["nom"] = item.findChildren(class_ = 'underline')[0].get_text()
                                    points = (item.findChildren(class_ = 'points')[0].get_text())
                                    indexSpacePoints = points.find(" ")
                                    points = points[:indexSpacePoints]
                                    OneNage["points"] = points
                                    
                                    Split = []
                                    if item.findChildren(class_ = 'temps_sans_tps_passage') != []:
                                        #print (item.findChildren(class_ = 'temps_sans_tps_passage')[0].get_text())
                                        temps = (item.findChildren(class_ = 'temps_sans_tps_passage')[0].get_text())
                                        temps = re.sub("\t|\r|\n","",temps)
                                        OneNage["temps"] = temps
                                    elif item.findChildren(class_ = 'temps') != []:
                                        #print (item.findChildren(class_ = 'temps')[0].findChildren(class_ = 'tooltip')[0].get_text())
                                        temps = (item.findChildren(class_ = 'temps')[0].findChildren(class_ = 'tooltip')[0].get_text())
                                        temps = re.sub("\t|\r|\n","",temps)
                                        #ON CHERCHE LE TEMPS GLOBAL
                                        
                                        indexSpace = temps.find(" ")
                                        TempsGlobal = temps[:indexSpace]
                                        OneNage["temps"] = TempsGlobal
                                        
                                        
                                        #ON CHERCHE LES TEMPS DE PASSAGES
                                        if item.findChildren(class_ = 'temps')[0].findChildren(class_ = 'split') == [] :
                                            print("ERREUR RECHERCHE DE TEMPS")
                                        else :
                                            SplitTab = item.findChildren(class_ = 'temps')[0].findChildren(class_ = 'split')[0].findChildren("tr")

                                            for s in SplitTab:
                                                OneSplit = {}
                                                OneSplit["distance"] = s.findChildren(class_ = 'distance')[0].get_text()
                                                OneSplit["temps"] = s.findChildren(class_ = 'split')[0].get_text()
                                                OneSplit["lap"] = s.findChildren(class_ = 'lap')[0].get_text()
                                                OneSplit["relay"] = s.findChildren(class_ = 'relay')[0].get_text()

                                                Split.append(OneSplit)

                                    #print (item.findChildren(class_ = 'resStructureDetailPlace')[0].get_text())

                                    OneNage["split"] = Split
                                    OneNage["place"] =  (item.findChildren(class_ = 'resStructureDetailPlace')[0].get_text())
                                    nages.append(OneNage)
                                    objet["nages"] = nages
                                    OneNage = {}
                        #print(Database)
                        nageurs.append(objet)
                        compete["nageurs"] = nageurs
                        Database.append(compete)
                        print(" ")
                        
                                    
                        file = "dataResult.json"
                        entry = {'carl': 33}  
                        #jsonFile = open("data.json", "w")
                        with open(file,"r+") as file :
                            data = json.load(file)
                            data.append(Database)
                            file.seek(0)
                            json.dump(data, file, indent=4,)
                            
                        #export = json.dumps(Database,indent=4)
                        #print (export)
                        #export = "dataR = "+export
                        #jsonFile = open("dataResult.json", "r+")
                        #jsonFile.write(export)
                        #jsonFile.close()
    if isTOEC == 0 :
        print("########### TOEC NOT PRESENT #############")
    return "Function done with succes"

        
    
    
        
        
##Init Json 

jsonFile = open("data.json", "w")
jsonFile.write('[{"init": 0}]')
jsonFile.close()
jsonFile = open("dataResult.json", "w")
jsonFile.write('[{"init": 0}]')
jsonFile.close()

#Main loop

for item in RegionalLink:
    lien = getProglink(item)
    if lien != None :
        lienstruct = getstructlink(lien)
        if lienstruct != None :
            
            liste=ListParticipants(lienstruct)
            linky = getsdetailstructlink("43203100195", liste)
            
            result = getresult("43203100195",liste,item)
            if result == None :
                print("error, structur not present")
            print(result)
            ## NUM DU TOEC : 43203100195"
        
        
        
        
        
filename = "data.json"
with open(filename, "r+") as file:
    data = json.load(file)

    export = json.dumps(data,indent=4)
    
export = "data = "+export
jsonFile = open("data.js", "w")
jsonFile.write(export)
jsonFile.close()

filename = "dataResult.json"
with open(filename, "r+") as file:
    data = json.load(file)

    export = json.dumps(data,indent=4)
    
export = "dataR = "+export
jsonFile = open("dataResult.js", "w")
jsonFile.write(export)
jsonFile.close()
print("######################################END OF PROGRAM #########################")
#print(RegionalLink)




########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######


########### TOEC NOT PRESENT #############
Function done with succes
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!######



########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######
######!!!!!######  Impossible to get data, web page different from standars ########!!!!!!!!!#######


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


###################TOEC FI



########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


########### TOEC NOT PRESENT #############
Function done with succes


## Display the data :

In [10]:
import json
from os import path

with open("dataResult.json", 'r') as f:
    data = f.read()
    json_data = json.loads(data)
    
print(json.dumps(json_data, indent=4))

[
    {
        "init": 0
    },
    [
        {
            "infos": {
                "nom": "Interclubs Avenirs - 25 m"
            },
            "nageurs": [
                {
                    "nages": [
                        {
                            "nom": "4x50 Nage Libre Dames S\u00e9ries",
                            "points": "747",
                            "split": [],
                            "place": "1\u00e8re"
                        },
                        {
                            "nom": "4x50 Nage Libre Dames S\u00e9ries",
                            "points": "549",
                            "split": [],
                            "place": "3e"
                        },
                        {
                            "nom": "4x50 Nage Libre Dames S\u00e9ries",
                            "points": "309",
                            "split": [],
                            "place": "6e"
                        },
                      